# Implementation of RFC 4226 - HOPT Algorithm
___HMAC based One Time Password___

...in python

* [RFC 4226](https://tools.ietf.org/html/rfc4226)
* [Wikipedia](https://en.wikipedia.org/wiki/HMAC-based_One-time_Password_algorithm)

In [1]:
import hashlib
import hmac
import base64

In [2]:
# Control excessive output to console
debug = True
def dbg(data):
    if (debug):
        print(data)

In [3]:
# Prepare Counter - convert integer to byte
def get_counter(counter):
    return counter.to_bytes(8, byteorder='big')

In [4]:
### Define SharedSecret, Block size, hashing algorithm, HOTP length
hash_algo = "sha1"
B = 64
counter = 1
shared_secret = b'BASE32SECRET3232'
# OTP Length
Digits = 6
# Google Authenticator Compatibility (BASE-32)
key=base64.b32decode(shared_secret)
dbg("Key Base32 Decode :")
dbg(key)

Key Base32 Decode :
b'\x08$M\xeaD\x14I=\xebz'


In [5]:
### Implement the HMAC Algorithm. For details see the rfc2104.ipynb at
# https://github.com/lordloh/OPT_algorithms/blob/master/rfc2104.ipynb

def my_hmac(key, message):
    trans_5C = bytes((x ^ 0x5C) for x in range(256))
    trans_36 = bytes((x ^ 0x36) for x in range(256))
    K_zpad=key.ljust(B,b'\0')    
    K_ipad=K_zpad.translate(trans_36)
    K_opad=K_zpad.translate(trans_5C)
    hash1 = hashlib.new(hash_algo, K_ipad+message).digest()
    hmac_hash = hashlib.new(hash_algo, K_opad + hash1).digest()
    return hmac_hash

In [6]:
### Dynamic Truncation
def dynamic_truncate(b_hash):
    dbg ("\n***** DYNAMIC TRUNCATION *****")
    hash_len=len(b_hash)
    int_hash = int.from_bytes(b_hash, byteorder='big')
    offset = int_hash & 0xF
    dbg ("offset = Lowest 4 bits (nibble) of hash = " + hex(int_hash & 0xF)+" = "+str(offset))
    dbg ("Get hex digits (nibbles) from digit #"+str(offset)+" to digit #"+str(offset+3))
    dbg ("Digit 0 is the most significat nibble of the hash.")
    # Geterate a mask to get bytes from left to right of the hash
    n_shift = 8*(hash_len-offset)-32
    MASK = 0xFFFFFFFF << n_shift
    #dbg ("\nTruncate MASK : "+hex(MASK).rjust(hash_len*2,"0"))
    hex_mask = "0x"+("{:0"+str(2*hash_len)+"x}").format(MASK)
    dbg ("\nHash            : 0x"+b_hash.hex())
    dbg ("Truncate MASK   : "+hex_mask+"\n")
    # Get rid of left zeros
    P = (int_hash & MASK)>>n_shift
    dbg ("Truncated hash (hex) : "+hex(P))
    dbg ("Truncated hash (int) : "+str(P))
    # Return only the lower 31 bits
    LSB_31 = P & 0x7FFFFFFF
    dbg ("31 LSB bits of truncated hash (hex) : "+hex(LSB_31))
    dbg ("31 LSB bits of truncated hash (int) : "+str(LSB_31))
    return LSB_31

The byte # of the HMAC hash are interpreted as - 


    -------------------------------------------------------------
    | Byte Number                                               |
    -------------------------------------------------------------
    |00|01|02|03|04|05|06|07|08|09|10|11|12|13|14|15|16|17|18|19|
    -------------------------------------------------------------

In [7]:
# function wrapper to run the HOTP algorithm multiple times for different counter value
def generate_HOTP(counter):
    C = get_counter(counter)
    
    dbg("Counter (int)     : "+str(counter)+"\nCounter (8 bytes) :")
    dbg(C)

    hmac_hash = my_hmac(key,C)
    dbg("\nHMAC Hash (counter) : 0x" + hmac_hash.hex())

    trc_hash = dynamic_truncate(hmac_hash)
    
    # Adjust HOTP length
    HOTP = "{:06}".format(trc_hash % (10**Digits))
    
    dbg("\n***** ADJUST DIGITS *****\n"+str(trc_hash)+" % 10 ^ "+str(Digits)+"\nHOPT : "+HOTP)
    return HOTP

In [8]:
# Generate HOTP for counter = 0
myHOTP0=generate_HOTP(0)
# Generate for counter = 0..10 without a lot of output messages.
debug=False
myHOTPs=[(generate_HOTP(x)) for x in range(1,10)]

Counter (int)     : 0
Counter (8 bytes) :
b'\x00\x00\x00\x00\x00\x00\x00\x00'

HMAC Hash (counter) : 5a9b22b8161f637bb9977fc56f56f921a93029b1

***** DYNAMIC TRUNCATION *****
offset = Lowest 4 bits (nibble) of hash = 0x1 = 1
Get hex digits (nibbles) from digit #1 to digit #4
Digit 0 is the most significat nibble of the hash.

Hash            : 0x5a9b22b8161f637bb9977fc56f56f921a93029b1
Truncate MASK   : 0x00ffffffff000000000000000000000000000000

Truncated hash (hex) : 0x9b22b816
Truncated hash (int) : 2602743830
31 LSB bits of truncated hash (hex) : 0x1b22b816
31 LSB bits of truncated hash (int) : 455260182

***** ADJUST DIGITS *****
455260182 % 10 ^ 6
HOPT : 260182


In [9]:
myHOTPs.insert(0,myHOTP0)
print(myHOTPs)

['260182', '055283', '795760', '172916', '437628', '220505', '845989', '311663', '850732', '285195']


## Compare with pyOTP Implementation

In [10]:
# Python
import pyotp

In [11]:
hotp1=pyotp.HOTP(shared_secret)

In [12]:
# Generate 0..9 HOTP codes
pyHOTPs=[(hotp1.at(x)) for x in range(10)]


In [13]:
print(pyHOTPs)

['260182', '055283', '795760', '172916', '437628', '220505', '845989', '311663', '850732', '285195']
